In [1]:
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
import datetime
import random
from scipy import stats
import math

#Avoiding Type 3 fonts in matplotlib plots
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.family'] = 'Arial'
pd.options.display.max_rows = 4000

In [2]:
font = {'size'   : 15}

matplotlib.rc('font', **font)
matplotlib.rc('lines', linewidth=2.0)
matplotlib.rc('lines', markersize=8)

# Modelling long - distance public trips
In this section we present the tools we used to derive the best-fitting amplified power-law process as well as the best-fitting truncated power-law for given emperical data-set


## Error function - sMAPE calculation

In order to determine whether a given CCDF $f$ describes the CCDF $g$ of our emperical data well, we use the symmetric mean absolute percentage error (sMAPE). That is, for a series of uniformly distributed sample points $S$, the function calculates the error as $\frac{2}{|S|}\sum_{x_i \in S} \dfrac{|f(x_i)-g(x_i)|}{ f(x_i)+g(x_i)}$.


In [3]:
''' Error function
Compute the distance(error) between the CCDFs of two given data sets

d1,d2: the two input data sets, given as a 1-D list of samples.
Out of each input, a CCDF is generated. The CCDFs are then compared according to above error metric.

max_d: the empirical trip length at which the CCDF reaches 10^{-3} on a log-log scale plot; this value defines the maximum range used for error calculation
num: the number of sampling points used in the error calculation

return: error value
'''

def cal_sMAPE(d1,d2,max_d,num):
    # Check input types
    if not (isinstance(d1, (np.ndarray, list, pd.Series)) and isinstance(d2, (np.ndarray, list, pd.Series))):
        raise ValueError('Wrong: Data type is not ndarray, List, or pd.Series')

    # Convert lists or pd.Series to numpy arrays for efficient computation
    d1 = np.array(d1)
    d2 = np.array(d2)

    # re-sampling
    # given the point x value and whole data, calculate the point of y value in CCDF
    sorted_d1 = np.sort(d1)
    y_d1=1- np.linspace(0, 1, len(sorted_d1), endpoint=False)

    sorted_d2 = np.sort(d2)
    y_d2=1- np.linspace(0, 1, len(sorted_d2), endpoint=False)
    
    # Create interpolation functions for the CCDFs
    interp_d1 = interp1d(sorted_d1, y_d1, bounds_error=False,fill_value=0)
    interp_d2 = interp1d(sorted_d2, y_d2, bounds_error=False,fill_value=0)

    
    # Find the overlapping range of the two distributions
    min_d = max(np.min(d1), np.min(d2))
    
    # Linearly spaced points for interpolation
    points = np.linspace(min_d, max_d, num)

    # Interpolate both distributions at the same points
    interp_y_d1 = interp_d1(points)
    interp_y_d2 = interp_d2(points)
    

    # Calculate the relative error in a vectorized way
    denom = (interp_y_d1 + interp_y_d2) / 2
    sum_error = np.abs(interp_y_d1 - interp_y_d2) / denom 
    error = np.nanmean(sum_error)
    
    return error


### Adapted sMAPE calculation for the exponentially truncated power-law

In our project, the exponentially truncated power-law does not provide an exact simulated trip length distribution but only a series of data points representing its CCDF, we slightly modify the sMAPE error function to better accommodate error calculation between the empirical data and the CCDF of the exponentially truncated power-law, unlike the error function used in the previous section.

In [4]:
''' Error function-----used to fit the power-law with the exponential truncation
Compute the distance between the CCDFs of the model and the empirical data.

d1: kind of CCDF values from the exponentially truncated power-law
d2: the regular 1-D samples as it in empirical data
Ds: the distribution of points for getting d1
max_d: the empirical trip length at which the CCDF reaches 10^{-3} on a log-log scale plot; this value defines the maximum range used for error calculation
num: the number of sampling points used in the error calculation

return: sMAPE error value
'''

def cal_sMAPE_exp(d1,d2,Ds,max_d,num):
    # Check input types
    if not (isinstance(d1, (np.ndarray, list, pd.Series)) and isinstance(d2, (np.ndarray, list, pd.Series))):
        raise ValueError('Wrong: Data type is not ndarray, List, or pd.Series')

    # Convert lists or pd.Series to numpy arrays for efficient computation
    d1 = np.array(d1)
    d2 = np.array(d2)

    sorted_d2 = np.sort(d2)
    y_d2=1- np.linspace(0, 1, len(sorted_d2), endpoint=False)

    # Create interpolation functions for the CCDFs,extended left=1, right=0
    interp_d1 = interp1d(Ds, d1, bounds_error=False,fill_value=(1,0)) # Ds is already sorted
    interp_d2 = interp1d(sorted_d2, y_d2, bounds_error=False,fill_value=(1,0))

    # Find the overlapping range of the two distributions
    min_d = max(np.min(Ds), np.min(d2))
    
    # Sampling method : Linear: # Linearly spaced points for interpolation
    points = np.linspace(min_d, max_d, num)

    # Interpolate both distributions at the same point
    interp_y_d1 = interp_d1(points)
    interp_y_d2 = interp_d2(points)


    # Calculate the relative error in a vectorized way
    denom = (interp_y_d1 + interp_y_d2) / 2
    sum_error = np.abs(interp_y_d1 - interp_y_d2) / denom 
    error = np.nanmean(sum_error)
        

    return error


## Get the maximum trip length for sMAPE calculation

Computes the last sample (largest trip length) to be considered in the sMAPE calculation. It corresponds to the trip length at which only  $10^{-3}$ propbabiliy mass remains in the CCDF.

In [5]:
''' Calculate the trip length value (max_d in the error calculation function) when the CCDF reaches 10^(-3) in a log-log scale CCDF plot, 
    which will be used to determine the maximum range of the samples used in the error calculation


d1: a 1-D dataset, like emipircal data in our project 
y_value: the exact value in y axis in a log-log scale CCDF plot

return: the value (as possible) of corresponding x value ----> max_d
'''
def get_xCCDF(d1, y_value=1e-3):
    # cal ccdf
    d1 = np.array(d1)
    sorted_d1 = np.sort(d1)
    y_d1 = 1 - np.linspace(0, 1, len(sorted_d1), endpoint=False)
    
    # target（log-log scale, 10^-3）
    target_y = y_value
    
    # Find the first position where y_d1 <= target_y (y_d1 is monotonically decreasing)
    idx = np.searchsorted(y_d1[::-1], target_y, side='left') # searchsorted need monotonically increasing, so inverse the y_d1
    idx = len(y_d1) - idx  # inverse index
    
    # Extract neighbouring points (make sure y1 > target_y > y2)
    x1, x2 = sorted_d1[idx-1], sorted_d1[idx]
    y1, y2 = y_d1[idx-1], y_d1[idx]
    
    # Linear interpolation in log-log space
    log_x = np.log10(x1) + (np.log10(target_y) - np.log10(y1)) / (np.log10(y2) - np.log10(y1)) * (np.log10(x2) - np.log10(x1))
    x_target = 10 ** log_x
    
    return x_target


## Fitting the amplified and truncated power-law models
Here we derive the parameters of the amplified power-law process that best describes the empirical data-set.
The values of the best-fitting process are then stored for later use.

### Distance-amplified power-law

This functions derives the best-fitting parameters for a distance amplified power-law via a grid search. This function can also be used to find the optimal parameters for the dynamically truncated power-law (by only considering combinations of parameters where $p=0$).

In [6]:
'''
Find the best-fitting parameter values for a distance-amplified power-law process
that follows the rules specified in the two previous blocks.

Input:
    combinable: an array of all parameter combinations to consider, error is null at the begining
    sampleSize: number of samples for finding the optimal parameters
    min_distance,max_distance: range of trip length to be considered
    maxdistance: used for dynamic method, usually equals to max_distance
    max_d: the empirical trip length at which the CCDF reaches 10^{-3} on a log-log scale plot; this value defines the maximum range used for error calculation
    num: the number of sampling points used in the error calculation
Return:
    full-filled combinale error
'''

def find_opt_params(combinable,sampleSize,min_distance,maxdistance,df_vec,max_d,num):
    tmp_max=maxdistance/2 # half of the maximum allowed distance for any trip
    n_samples = len(combinable)

    for i in range(n_samples):
        
        eps, p, alp, error, all_dist = combinable[i]  #note that (1+eps) is equal to amplification parameter C described in the paper
        dist = []
       
        # simulate each trip
        for n in range(sampleSize):
            distance=9999999
            # first determine the maximum distance the trip can have
            # we assume that the country (U.S and Germany) can be described by a circular shape of radius tmp_max
            
            # the trip is assigned a starting location inside the circle (selected uniformly at random)
            # the max_distance this trips is allowed to have is then the largest distance between the trips origin
            # and the edge of the circle.
    
            # instead of simulating this process for each trip, we consider the resulting distribution of these max_distance
            # values and sample from it as follows
            tmp=np.random.uniform(0,1)
            max_distance=np.sqrt(1-tmp)*tmp_max+tmp_max
            while(distance>=max_distance or distance<min_distance):

                # draw distance from the power-law distribution with parameter alp
                # amplification part
                #start with drawn power-law distance
                distance=np.random.uniform(0,1)**(-1/(alp-1))
                #compute how many times an amplifications occurs
                number=np.random.geometric(1-p)
                #amplify by factor (1+eps) each time
                distance=((1+eps)**(number-1))*distance

            dist.append(distance)

        combinable[i][4]=dist
        # error computation
        error=cal_sMAPE(dist,df_vec,max_d,num)
        combinable[i][3]=error

    
    return combinable

### Power-law with exponential truncation

The truncated exponential powerlaw given by parameters $\alpha$ and $\gamma$ is given by the density function $f(x) = C \cdot x^{-\alpha} \cdot e^{-x/\gamma}$. The constant $C > 0$ is used for normalization.

As we consider only data starting from a minimum distance, e.g., $100$ kilometers, we choose $C>0$ such that $\int_{100}^{\infty} f(x) dx = 1$.
To simplify the calculation, we instead approximate this integral as a sum $\sum_{i=100}^{2000} f(i)$, where we chose the upper limit large enough to only cause negigable inaccuracies.
The index values these sum runs over are given via the list `Ds` in our code.

In order to find the parameters of a truncated powerlaw distribution that best describes a data set, we consider a range of $\alpha$ and $\gamma$ values.
For any fixed combination of $\alpha$ and $\gamma$, first, $C$ is computed and then
the approximated CCDF $F(x) = \sum_{d \in Ds, d \geq x} f(d)$ is calculated.

As in the previous section, an error function is applied to assess the quality of each such CCDF and the best-fitting parameter combination is stored.

In [7]:
'''
Density function of the truncated power-law distribution
'''
def funcD(C,d,alpha,gamma):
    fD=C/d**alpha*(math.exp(-d/gamma))
    return fD

# This function is used as a helper function when computing C
def funcD_noC(d,alpha,gamma):
    fD_noC=1/d**alpha*(math.exp(-d/gamma))
    return fD_noC

In [8]:
'''
Finding optimal combination of parameters for power-law with exponential truncation

alphas, gammas, Ds: Sets of candidate parameter values used to identify the optimal exponentially truncated power-law.
data: the empirical dataset
max_d: the empirical trip length at which the CCDF reaches 10^{-3} on a log-log scale plot; this value defines the maximum range used for error calculation
num: the number of sampling points used in the error calculation

Return: optimal parameters for the exponentially truncated power-law, including alpha, gamma, C, and the corresponding error value.
'''
def get_results_PL_exp(alphas,gammas,Ds,data,max_d,num):
    results=[] # results contain alpha, gamma,C, error between the truncated power-law and emiprical data

    # for each combination of alpha and gamma
    for alpha in alphas:
        for gamma in gammas:
            # first compute the normalization constant C
            temp=0
            for d in Ds:
                temp+=funcD_noC(d,alpha,gamma)
            C=1/temp
    
            #compute the (approximate) CCDF F(d) of a truncated power-law
            #at several sample points given by Ds
            Fd=[0]*len(Ds)
            for idx,d in enumerate(Ds):
                for i in Ds[Ds>=d]:
                    Fd[idx]+=funcD(C,i,alpha,gamma)
            
            empircal_data=data.tolist()
            error=cal_sMAPE_exp(Fd,empircal_data,Ds,max_d,num)         
            # error=ccdf_sampling_error(df_vec,Ds,Fd)
            results.append([alpha,gamma,C,error])

    #convert results to dataframe
    results_truncated=pd.DataFrame(results, columns =['alpha', 'gamma', 'C','error']) 
    # save the optimal results
    opt_truncated=results_truncated.iloc[results_truncated.error.argmin(),:]

    return opt_truncated